<a href="https://colab.research.google.com/github/vishalvaka/Fake-Face-Detection/blob/main/project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
len(os.listdir('/content/sample_data/data/fake'))

FileNotFoundError: ignored

In [ ]:
import tensorflow
from tensorflow.keras.layers import Input, BatchNormalization, ReLU, Conv2D, Dense, MaxPool2D, AvgPool2D, GlobalAvgPool2D, Concatenate, Reshape, Lambda, GlobalAveragePooling2D
from tensorflow.keras.models import Model
import tensorflow.keras.backend as K
import tensorflow as tf
import random
import time
import os
import numpy as np

#from tensorflow.keras.utils import plot_model
#plot_model(model, show_shapes = True)

'''for i in range(2):
    out = get_pair()
    y.append(out[1])
    pairs.append(out[0])'''
#pairs, y = get_pair()


#np.array(pairs)
#pairs = np.swapaxes(pairs, 0, 1)

'for i in range(2):\n    out = get_pair()\n    y.append(out[1])\n    pairs.append(out[0])'

In [ ]:
import sys
print(sys.getrecursionlimit())
sys.setrecursionlimit(2000)

3000


In [ ]:
def bn_relu_conv(x, filters, kernel_size):
  x = BatchNormalization()(x)
  x = ReLU()(x)
  x = Conv2D(filters, kernel_size, padding='same')(x)
  return x

In [ ]:
  def dense_block(tensor, k, reps):
    for _ in range(reps):
      x = bn_relu_conv(tensor, k*4, 1)
      x = bn_relu_conv(x, k, 3)
      tensor = Concatenate()([tensor, x])
    return tensor

In [ ]:
def transition_layer(x, theta):
  f = int(tensorflow.keras.backend.int_shape(x)[-1]*theta)
  x = bn_relu_conv(x, f, 1)
  x = AvgPool2D(2, strides = 2, padding= 'same')(x)
  return x

In [ ]:
def dense_unit(x, k, num, channels, theta):
  for i in range(num-1):
    x = dense_block(x, k, channels)
    x = transition_layer(x, theta)

  x = dense_block(x, k, channels)

  return x

In [ ]:
def cffn():
  channels = [24, 30, 39, 63]
  num = [2, 3, 4, 2]
  k = 24
  theta = 0.5

  input = Input(shape = (64, 64, 3))
  x = Conv2D(48, 7, strides = 4, padding = 'same')(input)
  x = dense_unit(x, k, num[0], channels[0], theta)
  x = dense_unit(x, k, num[1], channels[1], theta)
  x = dense_unit(x, k, num[2], channels[2], theta)
  y = x
  x = dense_unit(x, k, num[3], channels[3], theta)
  y1 = x
  x = Dense(128)(x)
  y = Dense(128)(y)
  y1 = Dense(128)(y1)

  #pooledOutput = GlobalAveragePooling2D()(x)
  output = Concatenate()([x, Concatenate()([y, y1])])

  return Model(input, output)

In [ ]:
model = cffn()
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 64, 64, 3)]  0           []                               
                                                                                                  
 conv2d (Conv2D)                (None, 16, 16, 48)   7104        ['input_1[0][0]']                
                                                                                                  
 batch_normalization (BatchNorm  (None, 16, 16, 48)  192         ['conv2d[0][0]']                 
 alization)                                                                                       
                                                                                                  
 re_lu (ReLU)                   (None, 16, 16, 48)   0           ['batch_normalization[0][0]']

In [ ]:
def euclidean_distance(outputs):
    # unpack the vectors into separate lists
    (featsA, featsB) = outputs
    # compute the sum of squared distances between the vectors
    sumSquared = K.sum(K.square(featsA - featsB), axis=1, keepdims=True)
    # return the euclidean distance between the vectors
    return K.sqrt(K.maximum(sumSquared, K.epsilon()))

In [ ]:
def contrastive_loss(y, preds, margin=0.5):
    # explicitly cast the true class label data type to the predicted
    # class label data type (otherwise we run the risk of having two
    # separate data types, causing TensorFlow to error out)
    y = tf.cast(y, preds.dtype)
    # calculate the contrastive loss between the true labels and
    # the predicted labels
    squaredPreds = K.square(preds)
    squaredMargin = K.square(K.maximum(margin - preds, 0))
    loss = K.mean(y * squaredPreds + (1 - y) * squaredMargin)
    # return the computed contrastive loss to the calling function
    return loss

In [ ]:
imgA = Input(shape=(64, 64, 3))
imgB = Input(shape=(64, 64, 3))
featureExtractor = cffn()
featsA = featureExtractor(imgA)
featsB = featureExtractor(imgB)
# finally, construct the siamese network
distance = Lambda(euclidean_distance)([featsA[0][0], featsB[0][0]])
model = Model(inputs=[imgA, imgB], outputs=distance)

model.compile(loss=contrastive_loss, optimizer="adam", metrics = ['accuracy'])

#from tensorflow.keras.utils import plot_model
#plot_model(model, show_shapes = True)

In [ ]:
import os
i = 0
for img in os.listdir('/content/sample_data/data/real'):
  os.remove(os.path.join('/content/sample_data/data/real', img))
  i += 1
  if i == 100000:
    break

In [ ]:
from IPython.display import clear_output
for img in os.listdir('/content/sample_data/data/real'):
  if len(os.listdir('/content/sample_data/data/real')) == 100000:
    break
  os.remove(os.path.join('/content/sample_data/data/real', img))
  print(len(os.listdir('/content/sample_data/data/real')))
  clear_output(wait = True)

for img in os.listdir('/content/sample_data/data/fake'):
  if len(os.listdir('/content/sample_data/data/fake')) == 100000:
    break
  os.remove(os.path.join('/content/sample_data/data/fake', img))
  print(len(os.listdir('/content/sample_data/data/fake')))
  clear_output(wait = True)

In [ ]:
dataset = tf.keras.preprocessing.image_dataset_from_directory('/content/sample_data/data', image_size = (64, 64), batch_size = 16)

for images, labels in dataset:
  (pairs, labels) = make_pairs(images, labels)
  #print(pairs.shape)
  aug = tf.keras.preprocessing.image.ImageDataGenerator()
  model.fit(
	aug.flow([pairs[:,0], pairs[:, 1]], labels, batch_size=32),
	steps_per_epoch=len(pairs) // 32,
	epochs=2)

In [ ]:
#total_images = len(os.lisdir('/content/sample_data/data/real')) + len(os.listdir('/content/sample_data/data/fake'))

images = []
labels = []
for i in dataset:
  images.append(i[0])
  labels.append(i[1])
  print(np.array(images).shape)

  #model.fit([pairs[:,0], pairs[:, 1]], labels[:], batch_size=88, epochs=1)

In [ ]:
def correct(arr):
  out = []
  for i in arr:
    for j in i:
      out.append(j)
  return out

image_tensor = correct(np.load('/content/drive/MyDrive/images_rgb_all.npy', allow_pickle=True))
label_tensor = correct(np.load('/content/drive/MyDrive/labels_rgb_all.npy', allow_pickle=True))

In [ ]:
image_tensor = image_tensor[0:100000]
label_tensor= label_tensor[0:100000]

In [ ]:
import sklearn
images_shuffled, labels_shuffled = sklearn.utils.shuffle(image_tensor, label_tensor)

In [ ]:
np.array(images_shuffled).shape

(210415, 64, 64, 3)

In [ ]:
def get_label(label1, label2):
    if(label1 == label2):
        return 1
    else:
        return 0

In [ ]:

pairs = []
pairLabels = []
i = 0
while i<100000:
  pairs.append([images_shuffled[i], images_shuffled[i+1]])
  pairLabels.append(get_label(labels_shuffled[i], labels_shuffled[i+1]))
  i += 2

In [ ]:
np.array(pairs).shape

(50000, 2, 64, 64, 3)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(pairs, pairLabels, test_size = 0.1)

In [ ]:
X_train = np.array(X_train)
X_test = np.array(X_test)
y_test = np.array(y_test)
y_train = np.array(y_train)

In [ ]:
np.array([X_train[:,0], X_train[:, 1]][1]).shape

(84165, 64, 64, 3)

In [ ]:
from keras.callbacks import ModelCheckpoint
##checkpoint_path="/content/drive/MyDrive/checkpoint/cp-{epoch:04d}.ckpt"

#cp_callback = tf.keras.callbacks.ModelCheckpoint(
#    filepath=checkpoint_path,
#    verbose=1,
 #   save_weights_only=True,
 #   save_freq=1000)

aug = tf.keras.preprocessing.image.ImageDataGenerator()
model.fit_generator(aug.flow([X_train[:,0], X_train[:, 1]], y_train, batch_size=8),
    validation_data=([X_test[:,0], X_test[:, 1]], y_test), steps_per_epoch=len(X_train) // 8,
    epochs=2, use_multiprocessing = True, workers = 20)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/2
  97/5625 [..............................] - ETA: 7:05:33 - loss: 1353.5906 - accuracy: 0.4923

Process Keras_worker_ForkPoolWorker-14:
Process Keras_worker_ForkPoolWorker-2:
Process Keras_worker_ForkPoolWorker-3:
Process Keras_worker_ForkPoolWorker-19:
Process Keras_worker_ForkPoolWorker-15:
Process Keras_worker_ForkPoolWorker-5:
Process Keras_worker_ForkPoolWorker-1:
Process Keras_worker_ForkPoolWorker-18:
Process Keras_worker_ForkPoolWorker-9:
Process Keras_worker_ForkPoolWorker-6:
Process Keras_worker_ForkPoolWorker-10:
Process Keras_worker_ForkPoolWorker-8:
Process Keras_worker_ForkPoolWorker-17:
Process Keras_worker_ForkPoolWorker-12:
Process Keras_worker_ForkPoolWorker-11:
Process Keras_worker_ForkPoolWorker-13:
Process Keras_worker_ForkPoolWorker-4:
Process Keras_worker_ForkPoolWorker-16:
Process Keras_worker_ForkPoolWorker-20:
Process Keras_worker_ForkPoolWorker-7:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):

KeyboardInterrupt: ignored

In [ ]:
from keras.callbacks import ModelCheckpoint

#latest = tf.train.latest_checkpoint('/content/drive/MyDrive/checkpoint/cp.ckpt')

model.load_weights('./content/drive/MyDrive/checkpoint/')

NotFoundError: ignored

In [ ]:
(imagePairs, labelPairs) = make_pairs(np.array(image_tensor), np.array(label_tensor))

In [ ]:
aug = tf.keras.preprocessing.image.ImageDataGenerator()
model.fit(
	aug.flow([pairTrain[:,0], pairTrain[:, 1]], labelTrain, batch_size=64),
	validation_data=([pairTest[:,0], pairTest[:, 1]], labelTest), steps_per_epoch=len(pairTrain) // 64,
	epochs=2)

Epoch 1/2


In [ ]:
pairs = np.load('/content/drive/MyDrive/pairs.npy', allow_pickle = True)
labels = np.load('/content/drive/MyDrive/labels.npy', allow_pickle = True)

pairTrain = pairs[:9600]
pairTest = pairs[9601:]
labelTrain = labels[:9600]
labelTest = labels[9601:]

In [ ]:
pairs[0].shape

(2, 64, 64, 1)

In [ ]:
def make_pairs(images, labels):
    # initialize two empty lists to hold the (image, image) pairs and
    # labels to indicate if a pair is positive or negative
    pairImages = []
    pairLabels = []

    # calculate the total number of classes present in the dataset
    # and then build a list of indexes for each class label that
    # provides the indexes for all examples with a given label
    numClasses = len(np.unique(labels))
    idx = [np.where(labels == i)[0] for i in range(0, numClasses)]
    #print(idx)
        # loop over all images
    for idxA in range(len(images)):
        # grab the current image and label belonging to the current
        # iteration
        currentImage = images[idxA]
        label = labels[idxA]
        # randomly pick an image that belongs to the *same* class
        # label
        idxB = np.random.choice(idx[label])
        posImage = images[idxB]
        # prepare a positive pair and update the images and labels
        # lists, respectively
        pairImages.append([currentImage, posImage])
        pairLabels.append([1])
        # grab the indices for each of the class labels *not* equal to
        # the current label and randomly pick an image corresponding
        # to a label *not* equal to the current label
        #negIdx = np.where(labels != label)[0]
        negIdx = np.where(labels != label)[0]
        #print(negIdx)
        negImage = images[np.random.choice(negIdx)]
        #print(negImage)
        # prepare a negative pair of images and update our lists
        pairImages.append([currentImage, negImage])
        pairLabels.append([0])
    # return a 2-tuple of our image pairs and labels
    return (np.array(pairImages), np.array(pairLabels))

In [ ]:
idg1 = tf.keras.preprocessing.image.ImageDataGenerator()
gen1 = idg1.flow_from_directory('/content/sample_data/data',
                                shuffle=True,
                                class_mode='binary')

print(gen1[0][1])

Found 210415 images belonging to 2 classes.
[1. 0. 0. 1. 0. 1. 1. 1. 0. 0. 0. 1. 0. 1. 0. 1. 0. 1. 1. 1. 1. 1. 1. 0.
 0. 0. 0. 0. 1. 0. 1. 0.]


In [ ]:
import sklearn

idg1 = tf.keras.preprocessing.image.ImageDataGenerator()

gen1 = idg1.flow_from_directory('/content/sample_data/data',
                                shuffle=True,
                                class_mode='binary', target_size = (64, 64), batch_size = 16)


class JoinedGen(tf.keras.utils.Sequence):
    def __init__(self, input_gen1):
        self.gen1 = input_gen1

    def __len__(self):
        return len(self.gen1)

    def __getitem__(self, i):
        pairs, labels = make_pairs(gen1[i][0], gen1[i][1].astype(int))

        return [pairs[:, 0], pairs[:, 1]], labels

    def on_epoch_end(self):
        self.gen1.on_epoch_end()

Found 210415 images belonging to 2 classes.


In [ ]:
import sklearn
idg1 = tf.keras.preprocessing.image.ImageDataGenerator()

gen1 = idg1.flow_from_directory('/content/sample_data/data',
                                shuffle=True,
                                class_mode='binary', target_size = (64, 64), batch_size = 32)


class JoinedGen(tf.keras.utils.Sequence):
    def __init__(self, input_gen1):
        self.gen1 = input_gen1

    def __len__(self):
        return len(self.gen1)

    def __getitem__(self, i):
      images_shuffled, labels_shuffled = sklearn.utils.shuffle(gen1[i][0], gen1[i][1].astype(int))
      pairs = []
      pairLabels = []
      i = 0
      while i<len(images_shuffled):
        pairs.append([images_shuffled[i], images_shuffled[i+1]])
        pairLabels.append(get_label(labels_shuffled[i], labels_shuffled[i+1]))
        i += 2
      return [np.array(pairs)[:,0], np.array(pairs)[:, 1]], np.array(pairLabels)

    def on_epoch_end(self):
        self.gen1.on_epoch_end()

Found 210415 images belonging to 2 classes.


In [ ]:
tf.config.run_functions_eagerly(True)

mygen = JoinedGen(gen1)

model.fit(mygen, epochs = 4, steps_per_epoch=210415//32, use_multiprocessing = True, workers = 20)

Epoch 1/4


/usr/local/lib/python3.7/dist-packages/tensorflow/python/data/ops/dataset_ops.py:3704: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable.debug_mode()`.
  "Even though the `tf.config.experimental_run_functions_eagerly` "


   9/6575 [..............................] - ETA: 111:38:23 - loss: 2735.5876 - accuracy: 0.5000

Process Keras_worker_ForkPoolWorker-46:
Process Keras_worker_ForkPoolWorker-44:
Process Keras_worker_ForkPoolWorker-50:


KeyboardInterrupt: ignored

Process Keras_worker_ForkPoolWorker-54:
Process Keras_worker_ForkPoolWorker-49:
Process Keras_worker_ForkPoolWorker-45:
Process Keras_worker_ForkPoolWorker-42:
Process Keras_worker_ForkPoolWorker-53:
Process Keras_worker_ForkPoolWorker-41:


In [ ]:
pip install unrar
from unrar import rarfile
rar = rarfile.RarFile('/content/drive/MyDrive/fake/fake.rar')


SyntaxError: ignored

In [ ]:
pip install patool


     |████████████████████████████████| 81kB 4.0MB/s 


In [ ]:
import patoolib
patoolib.extract_archive("/content/drive/MyDrive/fake/fake.rar", outdir="/content/sample_data/data")

patool: Extracting /content/drive/MyDrive/fake/fake.rar ...
patool: running /usr/bin/unrar x -- /content/drive/MyDrive/fake/fake.rar
patool:     with cwd='/content/sample_data/data'
patool: ... /content/drive/MyDrive/fake/fake.rar extracted to `/content/sample_data/data'.


'/content/sample_data/data'

In [ ]:
patoolib.extract_archive("/content/drive/MyDrive/real.rar", outdir="/content/sample_data/data")

patool: Extracting /content/drive/MyDrive/real.rar ...
patool: running /usr/bin/unrar x -- /content/drive/MyDrive/real.rar
patool:     with cwd='/content/sample_data/data'
patool: ... /content/drive/MyDrive/real.rar extracted to `/content/sample_data/data'.


'/content/sample_data/data'

In [ ]:
import numpy as np
import os
import cv2
import matplotlib.pyplot as plt
import random
from IPython.display import clear_output
import time

DATADIR = '/content/sample_data/data'
CATEGORIES = ['real2', 'fake2']
IMG_SIZE = 64

def create_training_data():
  training_data = np.array([])
  img_array = np.empty(shape = (64, 64, 3))
  for category in CATEGORIES:
    i = 0
    path = os.path.join(DATADIR, category)
    class_num = CATEGORIES.index(category)
    for img in os.listdir(path):
      if i == 100000:
        break
      image_array = cv2.imread(os.path.join(path, img))
      img_array = cv2.resize(image_array, (IMG_SIZE, IMG_SIZE))
      clear_output(wait=True)
      print('reading image from '+str(path)+'/'+str(img))
      training_data = np.append(training_data, img_array)
      i += 1
      print(i)

  np.save('training_data_rgb', training_data)

create_training_data()

reading image from /content/sample_data/data/fake2/F_PGN1_02678.png
3000


In [ ]:
np.load('training_data_rgb.npy', allow_pickle = True).shape

(73728000,)

In [ ]:
data = tf.keras.preprocessing.image.ImageDataGenerator()

train_generator = data.flow_from_directory(
        '/content/sample_data/data',
        target_size=(64, 64),
        batch_size=32,
        class_mode='binary')

model.fit(
        train_generator,
        steps_per_epoch=3000 // 32,
        epochs=50,
        #validation_data=validation_generator,
        #validation_steps=800
        )

Found 6000 images belonging to 2 classes.
Epoch 1/50


ValueError: ignored

In [ ]:
pairs = np.load('/content/drive/MyDrive/pairs.npy', allow_pickle=True)
labels = np.load('/content/drive/MyDrive/labels.npy', allow_pickle=True)



model.fit(
	[pairTrain[:, 0], pairTrain[:, 1]], labelTrain[:],
	validation_data=([pairTest[:, 0], pairTest[:, 1]], labelTest[:])
	batch_size=88,
	epochs=2)

(12000, 2, 64, 64, 1)

In [ ]:
len(os.listdir('/content/sample_data/data/real'))

102599

In [ ]:
datagen = tf.keras.preprocessing.image.ImageDataGenerator()

print(datagen.flow_from_directory('/content/sample_data/data', target_size = (64, 64), class_mode='binary'))

Found 210415 images belonging to 2 classes.
